# Volatility notebook
This notebook explores realized volatility in the interst rate swap market for USD and CAD contracts around the time of the implementation of the Dodd-Frank Clearing Mandate.  The notebook proceeds as follows:
- Load transaction level data for USD and CAD swaps contracts
- Filter data for the appropriate inclusion conditions
    - Tenors of 2-year, 5-year and 10-year
    - References CDOR or LIBOR as the reference rate
    - Currency is USD or CAD
    - Trade Time is not on a Saturday, Sunday or holiday
    - Code is 'TR' (trade)
    - Type is 'IRS-FIX-FLOAT'
- Calculate the realized volatility for USD and CAD contracts
    - If contracts are specified in basis points, it is divided by 100 to express
    rates as percentages
    - A column for 'Categorical Trade Time' is calculated.
    8:00 AM - 10:59 - morning
    11:00 AM - 13:59 - mid-day
    14:00 - 17:00 - afternoon
    17:00 - 07:59 - off hours
    - The log of the fixed rate is calculated
    - The data are grouped by currency, date, categorical trade time and tenor
    - The realized return as the first difference of the log fixed rate
    - The volatility is calculated as the standard deviation of the realized
    return
-  Check if there are differences-in-differences before and after the implementation of the Clearing Mandate

In [13]:
import pandas as pd
from math import log
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [14]:
data = pd.read_excel('data/volatility/combined_trade_data.xlsx')
# Convert 'Effective' and 'Maturity' columns to datetime format
data['Effective'] = pd.to_datetime(data['Effective'])
data['Maturity'] = pd.to_datetime(data['Maturity'])
data['Trade Time'] = pd.to_datetime(data['Trade Time'])


In [15]:
# Add some additional columns
data['Trade Date'] = data['Trade Time'].dt.date
# Log price
data['log_rate_1'] = data['Rate 1'].apply(lambda x: log(x) if x > 0 else None)
def categorize_trade_time(time):
    if time.hour >= 8 and time.hour < 11:
        return 'morning'
    elif time.hour >= 11 and time.hour < 14:
        return 'mid day'
    elif time.hour >=  14 and time.hour < 17:
        return 'afternoon'
    else:
        return 'off hours'
    
data['trade_time_categorical'] = data['Trade Time'].apply(categorize_trade_time)
data['Grp'] = data['Curr'].apply(lambda x: 1 if x == 'USD' 
                                 else (0 if x == 'CAD' else -1))
data['Grp'] = data['Grp'].astype(int)

# Define a function to check if the trade date falls within the specified periods
def in_period(trade_date):
    # Convert the trade date to string format for comparison
    date_str = trade_date.strftime('%Y-%m-%d')
    
    # Define the date ranges in string format for 2013
    ranges = [
        ('2013-03-11', '2013-03-22'),
        ('2013-06-10', '2013-06-21'),
        ('2013-09-09', '2013-09-20'),
    ]
    
    # Check if the date string falls within any of the ranges
    return any(start <= date_str <= end for start, end in ranges)

# Apply the function to create the new 'period' column
data['period'] = data['Trade Time'].apply(lambda x: 1 if in_period(x) else 0)

data['interaction'] = data['Grp'] * data['period']

In [16]:

# Filter for the stuff I want
data_filtered = data[(data['Type'] == 'IRS Fix-Float') &
     (data['CD'] == 'TR') &
     ((data['Leg 2'] == 'USD-LIBOR-BBA') | (data['Leg 2'] == 'CAD-BA-CDOR')) &
     ((data['Curr'] == 'USD') | (data['Curr'] == 'CAD')) &
     (data['Trade Time'].dt.dayofweek < 5) &
     (data['Trade Time'].dt.date != '2013-09-02') &
     (data['Trade Time'].dt.date != '2013-05-27') &
     (data['T'].isin(['1Y', '2Y', '3Y', '4Y', '5Y', '6Y', '7Y', '8Y', '9Y', '10Y', '15Y', '30Y']))]

In [17]:
data_filtered = data_filtered.sort_values(by=['Curr', 'T', 'Trade Date', 'Trade Time']) \
                             .groupby(['Curr', 'T', 'Trade Date']) \
                             .apply(lambda x: x.assign(return_1=x['log_rate_1'].diff())) \
                             .reset_index(drop=True)

/var/folders/v2/2g59sj7s7qbcgl_0l5zkx1gm0000gn/T/ipykernel_41195/1660616700.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data_filtered = data_filtered.sort_values(by=['Curr', 'T', 'Trade Date', 'Trade Time']) \


In [18]:
# Group by 'Curr', 'T', and 'Trade Date', then calculate the standard deviation of 'return_1'
new_data = data_filtered.groupby(['Curr', 'T', 'Trade Date'])['return_1'].std().reset_index()

# Rename the column containing the standard deviation to 'std_return_1'
new_data = new_data.rename(columns={'return_1': 'std_return_1'})

new_data['Grp'] = new_data['Curr'].apply(lambda x: 1 if x == 'USD' 
                                 else (0 if x == 'CAD' else -1))
new_data['period'] = new_data['Trade Date'].apply(lambda x: 1 if in_period(x) else 0)
new_data['interaction'] = new_data['Grp'] * new_data['period']


new_data.to_csv('data/volatility/irs_fix_float_volatility.csv', index=False)

In [19]:
X = new_data[['period', 'Grp', 'interaction']]
X = sm.add_constant(X)  # Adds a constant term to the predictors
Y = new_data['std_return_1']

model = sm.OLS(Y, X, missing='drop').fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           std_return_1   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     40.86
Date:                Sun, 07 Apr 2024   Prob (F-statistic):           8.55e-25
Time:                        21:09:12   Log-Likelihood:                -350.71
No. Observations:                 925   AIC:                             709.4
Df Residuals:                     921   BIC:                             728.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.1270      0.033      3.812      